# DATA COMBINING

In [1]:
import pandas as pd
import os
import csv
import numpy as np

### Combine clean data:

- Read the suicide data (all age group)

In [2]:
# Load the suicide dataset
suicide_data = pd.read_csv('./clean_data/age_standardized_suicide_rates.csv')

suicide_data = suicide_data.drop(suicide_data.columns[[0]], axis = 1)

# Extract the country columns from both datasets
suicide_countries = suicide_data['Country'].str.strip()
suicide_data

,Region,Country,Year,Sex,actual suicide rate,low suicide rate,high suicide rate
0,Americas,Antigua and Barbuda,2019,Male,0.00,0.00,0.00
1,Americas,Barbados,2019,Female,0.16,0.11,0.22
2,Americas,Barbados,2019,Both sexes,0.31,0.22,0.42
3,Americas,Antigua and Barbuda,2019,Both sexes,0.32,0.22,0.45
4,Americas,Barbados,2019,Male,0.49,0.34,0.65
...,...,...,...,...,...,...,...
10975,Africa,Cameroon,2000,Female,9.71,5.73,15.47
10976,Africa,Ghana,2000,Both sexes,9.75,6.46,14.30
10977,Africa,Seychelles,2000,Both sexes,9.76,6.60,13.46
10978,Europe,Montenegro,2000,Female,9.82,7.22,12.89


- Read the suicide data (groupby age)

In [3]:
# Load the suicide dataset
suicide_age_data = pd.read_csv('./clean_data/suicide_rate_by_age.csv')

suicide_age_data = suicide_age_data.drop(suicide_age_data.columns[[0]], axis = 1)

# Extract the country columns from both datasets
suicide_age_countries = suicide_age_data['Country'].str.strip()
suicide_age_data

,Region,Country,Year,Sex,Age,suicide rate
0,Americas,Antigua and Barbuda,2019,Both sexes,15-24 years,0.00
1,Americas,Antigua and Barbuda,2019,Both sexes,25-34 years of age,0.00
2,Americas,Antigua and Barbuda,2019,Both sexes,35-44 years of age,0.00
3,Americas,Antigua and Barbuda,2019,Both sexes,45-54 years of age,0.00
4,Americas,Antigua and Barbuda,2019,Both sexes,65-74 years of age,0.00
...,...,...,...,...,...,...
4387,Africa,Burundi,2019,Male,75-84 years of age,97.84
4388,Europe,North Macedonia,2019,Male,85+ years,98.13
4389,Europe,Lithuania,2019,Male,85+ years,98.72
4390,Africa,Ethiopia,2019,Male,75-84 years of age,99.05


- Read and combine the temperature data

In [4]:
# Load the temperature dataset
temperature_data = pd.read_csv('./clean_data/country_temperature.csv')
temperature_data = temperature_data.drop(temperature_data.columns[[0]], axis = 1)

# Extract the country columns from both datasets
temperature_countries = temperature_data['Country'].str.strip()

# Finding the common countries
common_countries = set(suicide_countries).intersection(set(temperature_countries))

# Filter both datasets to include only the common countries
filtered_suicide_data = suicide_data[suicide_data['Country'].isin(common_countries)].reset_index(drop = True)
filtered_temperature_data = temperature_data[temperature_data['Country'].isin(common_countries)].reset_index(drop = True)

# Merge the datasets based on the country names and Year
merged_data_with_temperature = pd.merge(filtered_suicide_data, filtered_temperature_data, on = ['Country', 'Year'], how = 'inner')

# Save the merged data to a CSV file
merged_data_with_temperature.to_csv('./combine_data/merged_suicide_temperature_data.csv', index = False)

# Print confirmation
print("Data has been saved as 'merged_suicide_temperature_data.csv'.")
merged_data_with_temperature.head()

Data has been saved as 'merged_suicide_temperature_data.csv'.


,Region,Country,Year,Sex,actual suicide rate,low suicide rate,high suicide rate,AvgTemperature
0,Eastern Mediterranean,Kuwait,2019,Female,0.74,0.50,1.07,77.141370
1,Eastern Mediterranean,Jordan,2019,Female,0.93,0.61,1.37,62.505479
2,Americas,Honduras,2019,Female,0.98,0.53,1.69,68.827671
3,Americas,Panama,2019,Female,0.98,0.64,1.43,77.653151
4,Eastern Mediterranean,Oman,2019,Female,1.12,0.66,1.78,63.024161


- Combine the GDP data

In [5]:
# Load the GDP dataset
gdp_data = pd.read_csv('./raw_data/gdpdata.csv')
gdp_data = gdp_data.drop('Sr.No', axis = 1)

# Reshape the GDP dataset so that the GDP columns (years) are stacked as rows
gdp_data = pd.melt(gdp_data, id_vars = ["Country"], var_name = "Year", value_name = "GDP")

# Standardize country names using a mapping
country_name_mapping = {
    "Ivory Coast": "Côte d'Ivoire",
    "Viet Nam": "Vietnam",
    "Türkiye": "Turkey",
    "Iran (Islamic Republic of)": "Iran",
    "Republic of Korea": "South Korea",
    "Bolivia (Plurinational State of)": "Bolivia",
    "Lao People's DR": "Laos",
    "Côte d'Ivoire": "Ivory Coast",
}

# Apply the mapping to both datasets
gdp_data['Country'] = gdp_data['Country'].replace(country_name_mapping)
merged_data_with_temperature['Country'] = merged_data_with_temperature['Country'].replace(country_name_mapping)

# Change int64 type to object
merged_data_with_temperature['Year'] = merged_data_with_temperature['Year'].astype(str)

# # Merge the GDP data (for all years) with the final merged dataset
merged_data_with_gdp = pd.merge(merged_data_with_temperature, gdp_data, on = ['Country', 'Year'], how = 'inner')

# # Save the final merged data with all GDP years to a CSV file
merged_data_with_gdp.to_csv('./combine_data/merged_data_with_gdp.csv', index = False)

# # Print confirmation
print("Data has been saved as 'merged_data_with_gdp.csv'.")
merged_data_with_gdp.head()

Data has been saved as 'merged_data_with_gdp.csv'.


,Region,Country,Year,Sex,actual suicide rate,low suicide rate,high suicide rate,AvgTemperature,GDP
0,Eastern Mediterranean,Kuwait,2019,Female,0.74,0.50,1.07,77.141370,30666.220367
1,Eastern Mediterranean,Jordan,2019,Female,0.93,0.61,1.37,62.505479,4159.671442
2,Americas,Honduras,2019,Female,0.98,0.53,1.69,68.827671,2519.367423
3,Americas,Panama,2019,Female,0.98,0.64,1.43,77.653151,16472.831755
4,Eastern Mediterranean,Oman,2019,Female,1.12,0.66,1.78,63.024161,19132.150765


- Combine the population data

In [6]:
# Load the previously merged dataset and the population data
merged_data_with_population = pd.read_csv('./combine_data/merged_data_with_gdp.csv')
population_data = pd.read_csv('./raw_data/world_bank_population_data_1970_to_2022.csv')

# Rename the country column in population_data
population_data.rename(columns = {'country': 'Country', 'date': 'Year'}, inplace = True)

merged_data_with_gdp['Year'] = merged_data_with_gdp['Year'].astype(str)
population_data['Year'] = population_data['Year'].astype(str)

# Merge the population data with the reshaped GDP data on country and year
merged_data_with_population = pd.merge(merged_data_with_gdp, population_data, on = ['Country', 'Year'], how = 'inner')

# Because there is a country has a extreme value, we need to romove first
merged_data_with_population = merged_data_with_population.drop(merged_data_with_population[merged_data_with_population['Country'] == 'Malawi'].index)

# Save the final merged data with population to a CSV file
merged_data_with_population.to_csv('./combine_data/merged_data_with_population.csv', index = False)

# Print confirmation
print("Data has been saved as 'merged_data_with_population.csv'.")
merged_data_with_population['Year']

Data has been saved as 'merged_data_with_population.csv'.


0       2019
1       2019
2       2019
3       2019
4       2019
        ... 
4858    2000
4859    2000
4860    2000
4861    2000
4862    2000
Name: Year, Length: 4830, dtype: object

- Read and combine the happiness index data (2005 - 2020)

In [7]:
happiness_data = pd.read_csv('./clean_data/happiness_index.csv')
happiness_data = happiness_data.drop(happiness_data.columns[[0]], axis = 1)
happiness_data['Year'] = happiness_data['Year'].astype(str)
merged_data_with_happiness = pd.merge(merged_data_with_population , happiness_data, on = ['Country', 'Year'], how = 'inner')

# Save the final merged data with population to a CSV file
merged_data_with_happiness.to_csv('./combine_data/merged_data_with_happiness.csv', index = False)

# Print confirmation
print("Data has been saved as 'merged_data_with_happiness.csv'.")
merged_data_with_happiness.head()

Data has been saved as 'merged_data_with_happiness.csv'.


,Region,Country,Year,Sex,actual suicide rate,low suicide rate,high suicide rate,AvgTemperature,GDP,Population,Happiness Index
0,Eastern Mediterranean,Kuwait,2019,Female,0.74,0.50,1.07,77.141370,30666.220367,4441100.0,6.106
1,Eastern Mediterranean,Jordan,2019,Female,0.93,0.61,1.37,62.505479,4159.671442,10698683.0,4.453
2,Americas,Honduras,2019,Female,0.98,0.53,1.69,68.827671,2519.367423,9958829.0,5.930
3,Americas,Panama,2019,Female,0.98,0.64,1.43,77.653151,16472.831755,4232532.0,6.086
4,Eastern Mediterranean,Oman,2019,Female,1.12,0.66,1.78,63.024161,19132.150765,4602768.0,6.853


In [8]:
merged_data = pd.read_csv('./combine_data/merged_data_with_happiness.csv')
merged_data = merged_data.pivot(index=['Region', 'Country', 'Year', 'AvgTemperature', 'Happiness Index', 'GDP', 'Population'], columns=['Sex'], values='actual suicide rate').reset_index()
merged_data.to_csv('./combine_data/merged_data.csv', index = False)
merged_data

Sex,Region,Country,Year,AvgTemperature,Happiness Index,GDP,Population,Both sexes,Female,Male
0,Africa,Algeria,2005,62.913425,5.466833,3131.328300,32956690.0,3.82,2.80,4.83
1,Africa,Algeria,2006,64.930411,5.466833,3500.134528,33435080.0,3.65,2.66,4.63
2,Africa,Algeria,2007,63.166849,5.466833,3971.803658,33983827.0,3.46,2.51,4.41
3,Africa,Algeria,2008,63.532240,5.466833,4946.563793,34569592.0,3.31,2.40,4.22
4,Africa,Algeria,2009,64.259726,5.466833,3898.478923,35196037.0,3.15,2.29,4.02
...,...,...,...,...,...,...,...,...,...,...
1136,Western Pacific,Singapore,2015,81.980328,6.033000,54512.830319,5535002.0,7.56,4.98,10.03
1137,Western Pacific,Singapore,2016,81.303005,6.378000,55853.279583,5607283.0,7.30,5.16,9.34
1138,Western Pacific,Singapore,2017,81.988219,6.375000,59549.584598,5612253.0,7.25,5.07,9.45
1139,Western Pacific,Singapore,2018,80.348219,6.378000,64815.012628,5638676.0,7.76,5.00,10.37


# DATA PREPARATION

In [9]:
# Load the dataset
dataset = pd.read_csv('./combine_data/merged_data.csv')

# 1. Removing duplicates
dataset.drop_duplicates(inplace = True)

# 2. Handling missing data for numerical columns only
# Select numerical columns
numerical_columns = dataset.select_dtypes(include = [np.number]).columns

# Fill missing values with the mean for numerical columns
dataset[numerical_columns] = dataset[numerical_columns].fillna(dataset[numerical_columns].mean())

# 3. Checking data quality and attribute types
missing_values = dataset.isnull().sum()
data_types = dataset.dtypes
print("Missing Values:\n", missing_values)
print("Data Types:\n", data_types)

dataset.sort_values('Country')
dataset

# # Save the cleaned dataset
dataset.to_csv('./cleaned_final_dataset.csv', index = False)

# # Print confirmation
print("Dataset has been cleaned and saved as 'cleaned_final_dataset.csv'")

Missing Values:
 Region             0
Country            0
Year               0
AvgTemperature     0
Happiness Index    0
GDP                0
Population         0
Both sexes         0
Female             0
Male               0
dtype: int64
Data Types:
 Region              object
Country             object
Year                 int64
AvgTemperature     float64
Happiness Index    float64
GDP                float64
Population         float64
Both sexes         float64
Female             float64
Male               float64
dtype: object
Dataset has been cleaned and saved as 'cleaned_final_dataset.csv'


In [12]:
# Load the cleaned dataset
dataset = pd.read_csv('./cleaned_final_dataset.csv')

# 1. Identifying Attribute Types
categorical_columns = dataset.select_dtypes(include = ['object']).columns
numerical_columns = dataset.select_dtypes(include = [np.number]).columns

print("Categorical Columns:\n", categorical_columns)
print("Numerical Columns:\n", numerical_columns)

# 2. Data Statistics (Central Tendency, Dispersion)
data_statistics = dataset.describe()
print("Data Statistics:\n", data_statistics)

Categorical Columns:
 Index(['Region', 'Country'], dtype='object')
Numerical Columns:
 Index(['Year', 'AvgTemperature', 'Happiness Index', 'GDP', 'Population',
       'Both sexes', 'Female', 'Male'],
      dtype='object')
Data Statistics:
               Year  AvgTemperature  Happiness Index            GDP  \
count  1141.000000     1141.000000      1141.000000    1141.000000   
mean   2011.796670       61.710372         5.709026   17731.703703   
std       4.324636       20.092315         1.176341   21238.398575   
min    2005.000000      -97.900581         2.693000     151.188041   
25%    2008.000000       52.781967         4.812000    2218.197257   
50%    2012.000000       63.713661         5.770000    7085.121978   
75%    2016.000000       76.180328         6.650250   30666.220367   
max    2019.000000       85.438356         7.971000  103532.541013   

         Population   Both sexes       Female         Male  
count  1.141000e+03  1141.000000  1141.000000  1141.000000  
mean   